<a href="https://colab.research.google.com/github/kjg331/Analyze_Korean_comment/blob/main/Job_Korea_Crawling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By

import warnings
warnings.filterwarnings(action='ignore')

import pandas as pd

In [ ]:
driver = webdriver.Chrome("./chromedriver.exe")

In [ ]:
def login(driver):

    driver.get("https://www.jobkorea.co.kr/")

    # 로그인 페이지로 이동
    driver.find_element(By.XPATH, '//*[@id="devMyPage"]/ul/li[1]/a').click()
    # 아이디와 비밀번호 입력
    driver.find_element(By.ID, "M_ID").send_keys("Your ID")
    driver.find_element(By.ID, "M_PWD").send_keys("Your PW")
    # 로그인 버튼 클릭
    driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/div[2]/div[1]/form/fieldset/section[3]/button').click()

# a ~ b page의 url
def get_url(a, b):
    driver = webdriver.Chrome("./chromedriver.exe")
    driver.get("https://www.jobkorea.co.kr/")
    #login(driver)

    # 자기소개서 창
    driver.find_element(By.XPATH, '/html/body/div[2]/div/div[2]/div[1]/div/ul[1]/li[6]/a/span').click()
    driver.get("https://www.jobkorea.co.kr/starter/passassay")
    driver.maximize_window()
    driver.implicitly_wait(3)
    driver.find_element(By.XPATH, '/html/body/div[5]/div/button').click()

    # 30대그룹, 매출1000대, 대기업, 공공기관, 외국계 자소서 클릭
    for i in range(1, 6):
            driver.find_element(By.XPATH, f"/html/body/div[4]/div[2]/div[2]/div[2]/div/div[1]/div/dl[5]/dd/div/div[1]/div/ul[1]/li[{i}]/label/i").click()
    # 중견기업 자소서 클릭
    driver.find_element(By.XPATH, f"/html/body/div[4]/div[2]/div[2]/div[2]/div/div[1]/div/dl[5]/dd/div/div[1]/div/ul[2]/li[1]/label/i").click()

    # 필터 적용
    driver.find_element(By.XPATH, '//*[@id="container"]/div[2]/div[2]/div/div[2]/button').click()

    # 전문가 분석 필터
    driver.find_element(By.XPATH,'/html/body/div[4]/div[2]/div[2]/div[5]/div[1]/form/fieldset/div[1]/span[2]/label').click()

    urls = []
    for i in range(a, b):
        try:
            driver.get(f"https://www.jobkorea.co.kr/starter/PassAssay?FavorCo_Stat=0&Pass_An_Stat=1&OrderBy=0&EduType=0&WorkType=0&schCType=11%2C12%2C13&isSaved=0&Page={i}")
            driver.implicitly_wait(2)
            # url 추출하기
            com_list = driver.find_element(By.XPATH,"/html/body/div[4]/div[2]/div[2]/div[5]/ul")
            com_url = com_list.find_elements(By.TAG_NAME,'a') # 거기에서 a태그만 가져온다

            for url in com_url:
                if 'selfintroduction' in url.get_attribute('href'):
                    pass
                else:
                    urls.append(url.get_attribute("href"))
                    print(f"Page {i} is completed")
                    driver.implicitly_wait(3)
        except:
            print(f"Page{i}has a error")
            pass
    driver.close()

    return list(set(urls))

def get_content(url):
    driver = webdriver.Chrome("./chromedriver.exe")
    login(driver)
    driver.get(url)

    # Position name
    position_element = driver.find_element(By.XPATH, '/html/body/div[4]/div[2]/div[2]/div[1]/div[1]/h2/em')
    position = position_element.text[13:]

    # Question+Answer+Advcie
    qna_list = driver.find_element(By.CLASS_NAME, 'qnaLists')

    # Question
    question_list = []
    questions = qna_list.find_elements(By.TAG_NAME,'dt')

    for index in questions:
        question = index.find_element(By.CLASS_NAME,'tx')

        #태그가 펼쳐지지 않아서 크롤링이 안되는 경우
        if question.text=="":
            index.find_element(By.TAG_NAME,'button').click()
            question = index.find_element(By.CLASS_NAME,'tx')
            question_list.append(question.text)

        # 펼쳐져 있는 경우
        else:
            question_list.append(question.text)

    driver.implicitly_wait(3)

    # Answer
    answer_list = []
    answers = qna_list.find_elements(By.TAG_NAME,'dd')
    driver.implicitly_wait(3)

    for index in range(len(answers)):
        answer = answers[index].find_element(By.CLASS_NAME,'tx')

        if answer.text == "":
            questions[index].find_element(By.TAG_NAME,'button').click()
            answer=answers[index].find_element(By.CLASS_NAME,'tx')
            answer_element = answer.text
            idx = answer_element.find("글자수")
            answer_list.append(answer_element[:idx])
        else:
            answer_element = answer.text
            idx = answer_element.find("글자수")
            answer_list.append(answer_element[:idx])

    # Advice
    advice_list = []
    advices = answers # answers와 같은 Tag내에 있음
    for index in range(len(advices)):
        advice=advices[index].find_element(By.CLASS_NAME,'advice')

        if advice.text == "":
            questions[index].find_element(By.TAG_NAME,'button').click()
            advice=advices[index].find_element(By.CLASS_NAME,'advice')
            advice_list.append(advice.text)
        else:
            advice_list.append(advice.text)

    content_df = pd.DataFrame({'position':position,
                             'question':question_list,
                             'answer':answer_list,
                              'advice':advice_list})
    return content_df

In [ ]:
def question_prompot(position, question, answer):
    prompt_template = """### Question:
    아래 {position}의 자기 소개서에 대해 평가해주세요.

    {question}

    {answer}

    ### Answer:"""
    return(prompt_template.format(position=position, question=question, answer=answer))

In [ ]:
new_urls = get_url(135, 139)

Page135has a error
Page 136 is completed
Page 136 is completed
Page 136 is completed
Page 136 is completed
Page 136 is completed
Page 136 is completed
Page 136 is completed
Page 136 is completed
Page 136 is completed
Page 136 is completed
Page 136 is completed
Page 136 is completed
Page 136 is completed
Page 136 is completed
Page 136 is completed
Page 136 is completed
Page 136 is completed
Page 136 is completed
Page 136 is completed
Page 136 is completed
Page 136 is completed
Page 136 is completed
Page 136 is completed
Page 136 is completed
Page 136 is completed
Page 136 is completed
Page 136 is completed
Page 136 is completed
Page 136 is completed
Page 136 is completed
Page 136 is completed
Page 136 is completed
Page 136 is completed
Page 136 is completed
Page 136 is completed
Page 136 is completed
Page 136 is completed
Page 136 is completed
Page 136 is completed
Page 136 is completed
Page 136 is completed
Page 136 is completed
Page 136 is completed
Page 136 is completed
Page 136 is c

In [ ]:
pd.DataFrame({"url": urls}).to_csv("job_urls.csv", header = False, index = False)

In [ ]:
urls.to_csv("job_urls.csv", header = False, index = False)

In [ ]:
urls["url"][0]

'https://www.jobkorea.co.kr/starter/PassAssay/View/200641?Page=27&OrderBy=0&FavorCo_Stat=0&schCType=11%2C12%2C13&Pass_An_Stat=1'

In [ ]:
position = get_content(urls[2])

In [ ]:
examples = position.to_dict()
examples

{'position': {0: '품질관리자', 1: '품질관리자', 2: '품질관리자', 3: '품질관리자'},
 'question': {0: '삼성전자를 지원한 이유와 입사 후 회사에서 이루고 싶은 꿈을 기술하십시오.',
  1: '본인의 성장과정을 간략히 기술하되 현재의 자신에게 가장 큰 영향을 끼친 사건, 인물 등을 포함하여 기술하시기 바랍니다. (※작품속 가상인물도 가능)',
  2: '최근 사회이슈 중 중요하다고 생각되는 한가지를 선택하고 이에 관한 자신의 견해를 기술해 주시기 바랍니다.',
  3: '지원한 직무 관련 본인이 갖고 있는 전문지식/경험(심화전공, 프로젝트, 논문, 공모전 등)을 작성하고, 이를 바탕으로 본인이 지원 직무에 적합한 사유를 구체적으로 서술해 주시기 바랍니다.'},
 'answer': {0: '"차세대 예지 보전 설비기술 도입"\n\n21살 때, 삼성전자 화성캠퍼스 신설 현장에서 일하며 반도체 설비에 관한 관심을 키우게 되었습니다. 당시, 공장 내부 가스관 설치 현장에서 배관 시공을 간접적으로 경험하며 설비 유지 보수에 관심을 키우게 되었습니다. 아무리 완벽한 반도체 이론이 있더라도 원활한 설비 공급 없이는 제품을 구현할 수 없다는 점을 느꼈습니다. 삼성전자 반도체 연구소는 세계 최고의 기술을 연구하여 글로벌 메모리 반도체 1위 자리를 30년간 지켜내고 있습니다. 이런 설비기술의 선구자인 삼성전자에서 차세대 예지 보전 설비관리 특허보유라는 꿈을 이루고 싶습니다.좋은점 1\n\n먼저, 반도체 제조의 핵심인 단위공정 설비팀에서 근무하며 경험을 쌓고 싶습니다. 특히, 열역학적 지식이 필요한 산화 및 증착 공정 설비기술팀에서 근무하며 세계 최고의 전문가들로부터 설비 지식을 배우겠습니다. 또한, 삼성전자의 직무 심화 양성프로그램을 활용하여 설비 전문가로 성장하겠습니다. 이후 항상 현장에 관심을 두고 문제점을 파헤쳐 유지관리 프로세스를 개선하기 위해 노력하겠습니다. 특히, 차세대 예지 보전 기술도입을 통해 유지비용을 절감하고 예지 보전

In [ ]:
dataset = []
for i in range(len(examples)):
    question = question_prompot(examples['position'][i], examples['question'][i], examples['answer'][i])
    answer = examples['advice'][i]
    dataset.append({"question":question, "answer":answer})

In [ ]:
dataset[1]

{'question': '### Question:\n    아래 품질관리자의 자기 소개서에 대해 평가해주세요.\n\n    본인의 성장과정을 간략히 기술하되 현재의 자신에게 가장 큰 영향을 끼친 사건, 인물 등을 포함하여 기술하시기 바랍니다. (※작품속 가상인물도 가능)\n    \n    "시행착오를 통한 배움의 자세"\n\n학부 2학년 때 CATIA 프로그램을 이용해 지게차를 모델링 하며 새로운 배움에 대한 두려움을 떨쳐낸 경험이 있습니다. 처음 접하는 설계 프로그램이었기에 결과물을 완성할 수 있을지에 대한 막연한 두려움이 있었습니다. 특히, 지게차 체인 조립 과정에서 한계를 느꼈습니다. 하지만 체인 설계 방법을 다양하게 바꿔보는 시도를 통해 해결할 수 있었습니다. 이를 통해 배움에 있어 고비는 반드시 찾아오지만, 시행착오를 통해 얼마든지 극복할 수 있다는 자신감을 얻었습니다.\n\n이러한 자신감은 이듬해 기업 과제 프로젝트 중, 팀원들이 도전하기 꺼리는 Android Studio 프로그램 학습을 자발적으로 맡은 계기가 되었습니다. 당시, 기업에서 제작 중인 호신용 애플리케이션 개선을 의뢰받았습니다. 다들 전공 분야가 아닌 프로그램을 배우기 꺼려했지만, 할 수 있다는 자신감을 가지고 프로그램을 배우기로 나섰습니다. 수많은 시행착오를 통해 위치전송 서비스를 구현과 소비자 친화적인 UI 구성을 완성했습니다. 그 결과, 기업에서 저희 팀의 결과물을 채택했고 프로젝트를 성공적으로 마칠 수 있었습니다. 이렇듯 입사 후에도 배움에 대한 자신감을 통해 설비기술 전문가로 성장할 수 있다고 확신합니다. 포기하지 않고 꾸준히 탐구하여 설비의 전문적인 지식을 갖추어 최적의 설비 운영을 이루겠습니다.아쉬운점 1\n\n"창의력과 추진력을 통한 문제 해결 능력"\n\n설비기술 엔지니어에겐 불합리한 현상에 지식을 융합하여 문제점을 개선하는 창의력이 필요하다고 생각합니다. 학부 과정 동안 경진대회 15번을 참가하며 문제 해결 능력을 키웠습니다. 특히 4학년 시절 창의력과 탐구 의지를 

In [ ]:
from pprint import pprint
print(dataset[0])

{'question': '### Question:\n    아래 품질관리자의 자기 소개서에 대해 평가해주세요.\n\n    삼성전자를 지원한 이유와 입사 후 회사에서 이루고 싶은 꿈을 기술하십시오.\n    \n    "차세대 예지 보전 설비기술 도입"\n\n21살 때, 삼성전자 화성캠퍼스 신설 현장에서 일하며 반도체 설비에 관한 관심을 키우게 되었습니다. 당시, 공장 내부 가스관 설치 현장에서 배관 시공을 간접적으로 경험하며 설비 유지 보수에 관심을 키우게 되었습니다. 아무리 완벽한 반도체 이론이 있더라도 원활한 설비 공급 없이는 제품을 구현할 수 없다는 점을 느꼈습니다. 삼성전자 반도체 연구소는 세계 최고의 기술을 연구하여 글로벌 메모리 반도체 1위 자리를 30년간 지켜내고 있습니다. 이런 설비기술의 선구자인 삼성전자에서 차세대 예지 보전 설비관리 특허보유라는 꿈을 이루고 싶습니다.좋은점 1\n\n먼저, 반도체 제조의 핵심인 단위공정 설비팀에서 근무하며 경험을 쌓고 싶습니다. 특히, 열역학적 지식이 필요한 산화 및 증착 공정 설비기술팀에서 근무하며 세계 최고의 전문가들로부터 설비 지식을 배우겠습니다. 또한, 삼성전자의 직무 심화 양성프로그램을 활용하여 설비 전문가로 성장하겠습니다. 이후 항상 현장에 관심을 두고 문제점을 파헤쳐 유지관리 프로세스를 개선하기 위해 노력하겠습니다. 특히, 차세대 예지 보전 기술도입을 통해 유지비용을 절감하고 예지 보전 분야 특허 확보를 위해 노력하겠습니다. 세계 유일 종합반도체 연구소의 선행개발 엔지니어라는 자부심을 쌓고 최첨단 설비 전문가로 성장하여 종합 반도체 부분 세계 1위 목표를 함께 이루고 싶습니다.좋은점 2\n\n\n    ### Answer:', 'answer': '좋은점 1 어떻게 이 회사에서 일을 하게 되었는지에 대한 구체적인 설명은 없지만 지원 회사에서 일을 하면서 느낀 점과 그 내용을 바탕으로 지원동기를 작성한 점이 좋습니다.\n좋은점 2 이루고 싶은 꿈을 구체적으로 제시한 점이 좋고 지원동기에 일관되어 있다